## Perform Fisher's Exact Tests to test if differences in allele frequencies for population groups

In [161]:
# Import modules and packages

import os
from scipy.stats import fisher_exact
import pandas as pd
import itertools


In [147]:
# Set constants and functions 

home_path = str(os.path.dirname(os.getcwd()))
population_clusters = ["SUPER","SUB"]

gene_location_df = pd.read_csv(os.path.join(home_path, "Data_descriptions", "locations.csv"))
genes = gene_location_df.location_name

sample_info_df = pd.read_csv(os.path.join(home_path, "Data_descriptions", "samples.csv"))
sub_populations = sample_info_df.SUB.unique().tolist()
regional_classification = {"ACB":"ACB","ASW":"ASW","GWD":"WA","ESN":"WA","MSL":"WA","MbutiPygmy":"CA","BiakaPygmy":"CA","Mandenka":"WA","Yoruba":"WA","San":"KS","BantuSouthAfrica":"SA","BantuKenya":"EA","YRI":"WA","LWK":"EA"} # SA = South Africa, CA = Central Africa, WA = West Africa, EA = East Africa, ACB = African Carribean in Barbados, ASW = African American, KS = Khoi-San

def add_prefix_dataframe_col_names(dataframe, columns_list, prefix):
    rename_dict = dict()
    for line in columns_list:
        key = line
        if key not in rename_dict:
            rename_dict[key] = str()
        rename_dict[key] = prefix + "{}".format(line)
    dataframe = dataframe.rename(columns = rename_dict)
    return dataframe

In [4]:
# Import allele count data

collated_count_data = pd.DataFrame()
for gene in genes:
    for sub_population in sub_populations:
        allele_count_path = os.path.join(home_path,"Data", "SUB", "ALL_{}.{}.acount".format(gene, sub_population))
        if os.path.exists(allele_count_path):
            allele_count_df = pd.read_csv(allele_count_path, sep="\t")
            allele_count_df["REF_CTS"] = allele_count_df["OBS_CT"] - allele_count_df["ALT_CTS"]
            allele_count_df["GENE"] = gene
            allele_count_df["SUB_POP"] = sub_population
        collated_count_data = pd.concat([collated_count_data, allele_count_df]).drop(columns="OBS_CT")

collated_count_data

,#CHROM,ID,REF,ALT,ALT_CTS,REF_CTS,GENE,SUB_POP
0,13,rs1185300901,C,CT,0,188,COL4A1,ACB
1,13,rs552586867,C,G,0,188,COL4A1,ACB
2,13,rs59409892,C,G,19,169,COL4A1,ACB
3,13,rs535182970,G,C,1,187,COL4A1,ACB
4,13,rs56406633,A,G,1,187,COL4A1,ACB
...,...,...,...,...,...,...,...,...
158,21,rs981058453,C,T,0,104,OLIG2,ASW
159,21,rs182058038,T,A,1,103,OLIG2,ASW
160,21,rs7278343,A,G,21,83,OLIG2,ASW
161,21,rs151281307,AT,A,1,103,OLIG2,ASW


## Format allele count data in format suitable for Fisher's test

In [152]:
fishers_data = collated_count_data[collated_count_data.ID != "."] # Drop ambiguous variant IDs
fishers_data = fishers_data.drop_duplicates(["#CHROM","ID","REF","ALT","SUB_POP"]) # Drop duplicate entries
fishers_data = fishers_data.pivot(index=["#CHROM","ID","REF","ALT"], columns="SUB_POP", values=["ALT_CTS","REF_CTS"]) # Pivot data

# Separate alternate and reference count data into different dataframes to facilate renaming of count columns appropriately
fishers_data_alt = fishers_data[['ALT_CTS']].droplevel(level=0, axis=1).reset_index() 
fishers_data_ref = fishers_data[['REF_CTS']].droplevel(level=0, axis=1).reset_index()

fishers_data_alt = add_prefix_dataframe_col_names(fishers_data_alt, sub_populations,"ALT_CT_")
fishers_data_ref = add_prefix_dataframe_col_names(fishers_data_ref, sub_populations,"REF_CT_")

# Merge renamed alternate and reference count data
fishers_data_renamed = fishers_data_alt.merge(fishers_data_ref, on=["#CHROM","ID","REF","ALT"])
fishers_data_renamed

SUB_POP,#CHROM,ID,REF,ALT,ALT_CT_ACB,ALT_CT_ASW,ALT_CT_BantuKenya,ALT_CT_BantuSouthAfrica,ALT_CT_BiakaPygmy,ALT_CT_ESN,...,REF_CT_BiakaPygmy,REF_CT_ESN,REF_CT_GWD,REF_CT_LWK,REF_CT_MSL,REF_CT_Mandenka,REF_CT_MbutiPygmy,REF_CT_San,REF_CT_YRI,REF_CT_Yoruba
0,1,rs1000680496,G,A,0,0,0,0,0,0,...,42,206,231,184,166,40,24,12,234,42
1,1,rs1001911383,C,T,0,0,0,0,0,0,...,36,204,228,182,166,38,22,12,234,36
2,1,rs1002151986,A,C,0,0,0,0,0,0,...,42,206,232,184,166,40,24,12,234,42
3,1,rs1002155376,G,T,0,0,0,0,0,0,...,42,206,232,184,166,40,24,12,234,42
4,1,rs1002228547,G,C,1,0,0,0,0,0,...,42,206,232,184,166,40,24,12,234,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20302,21,rs972037300,C,G,0,0,0,0,0,0,...,42,206,232,184,166,40,24,12,234,42
20303,21,rs975220347,C,T,0,0,0,0,0,0,...,42,206,232,184,166,40,24,12,234,42
20304,21,rs981058453,C,T,0,0,0,0,0,0,...,42,206,232,184,166,40,24,12,234,41
20305,21,rs997790265,C,CG,0,0,0,0,0,0,...,42,204,232,182,166,38,24,12,234,42


## Perform Fisher's Exact test

In [ ]:
# Generate all possible combinations of populations for comparison
combinations = list(itertools.combinations(sub_populations,2))
combinations


In [ ]:
# Calculate fisher's exact odds ratios and p-values for each variant for the different population combinations
for combination in combinations:
    first_pop = combination[0]
    second_pop = combination[1]
    for index, row in fishers_data_renamed.iterrows():
        alt_1 = row["ALT_CT_{}".format(first_pop)]
        ref_1 = row["REF_CT_{}".format(first_pop)]
        alt_2 = row["ALT_CT_{}".format(second_pop)]
        ref_2 = row["REF_CT_{}".format(second_pop)]
        oddsratio, pvalue = fisher_exact([[alt_1,alt_2],[ref_1,ref_2]])
        print (oddsratio, pvalue)